In [20]:
GAMING_CSV_PATH = "../Jarir-scraper/jarir_gaming_pcs.csv"
LAPTOP_CSV_PATH = "../Jarir-scraper/jarir_laptops.csv"
TABLET_CSV_PATH = "../Jarir-scraper/jarir_tablets.csv"
twoin1_CSV_PATH = "../Jarir-scraper/jarir_twoin1_laptops.csv"
DESKTOPS_CSV_PATH = "../Jarir-scraper/jarir_desktops.csv"


csv_paths = [GAMING_CSV_PATH, LAPTOP_CSV_PATH, TABLET_CSV_PATH , twoin1_CSV_PATH , DESKTOPS_CSV_PATH]

import pandas as pd
from pathlib import Path
from typing import Dict, List

def build_product_type_first_map(csv_paths: List[str]) -> Dict[str, Dict[str, List[str]]]:
    """
    Read each CSV at the given file paths (each must have 'brand' and 'model' cols),
    and merge into one dict mapping:

        product_type → { brand → sorted list of unique models }.

    product_type will be one of:
      "gaming", "laptop", "tablet", "twoin1", "desktops"
    """
    nested_map: Dict[str, Dict[str, List[str]]] = {}

    for path in csv_paths:
        # 1) derive normalized product_type
        stem = Path(path).stem.lower()            # e.g. "jarir_gaming_pcs"
        if stem.startswith("jarir_"):
            stem = stem[len("jarir_"):]           # → "gaming_pcs"
        stem = stem.replace("_pcs", "")           # → "gaming"
        # singularize where needed:
        if stem.endswith("s") and stem not in ("desktops",):
            stem = stem[:-1]                      # "laptops"→"laptop", "tablets"→"tablet"
        product_type = stem                       # now one of gaming, laptop, tablet, twoin1, desktops

        # 2) read & group
        df = pd.read_csv(path, usecols=["brand", "model"])
        brand_map: Dict[str, List[str]] = {}
        for brand in df["brand"].dropna().unique():
            models = (
                df.loc[df["brand"] == brand, "model"]
                  .dropna()
                  .unique()
                  .tolist()
            )
            brand_map[brand] = sorted(models)

        nested_map[product_type] = brand_map

    return nested_map

product_type_map = build_product_type_first_map(csv_paths)
print(product_type_map.keys())
def retrieve_information_about_product_type(product_type: str) -> Dict[str, List[str]]:
    """
    These are the available product_type [gaming, laptop, tablet , twoin1 , desktops]
    Given a product type, returns a dict mapping each brand
    to the list of models available under that product_type.
    """
    return product_type_map.get(product_type, {})

dict_keys(['gaming', 'laptop', 'tablet', 'twoin1_laptop', 'desktops'])


{'Microsoft': ['Surface'], 'Lenovo': ['IdeaPad 3', 'IdeaPad Pro 5', 'IdeaPad Slim 3', 'IdeaPad Slim 5', 'ThinkBook 14', 'ThinkBook 14 G8', 'ThinkBook 16 G7', 'ThinkBook 16 G8', 'ThinkBook 16p', 'ThinkPad X9 14', 'ThinkPad X9 15', 'Yoga Pro 7', 'Yoga Pro 7 Aura Edition', 'Yoga Pro 9', 'Yoga Slim 6', 'Yoga Slim 7', 'Yoga Slim 7 Aura Edition', 'Yoga Slim 7i'], 'Acer': ['Aspire 3', 'Aspire 5', 'Aspire Go', 'Aspire Lite', 'Swift'], 'Huawei': ['MateBook', 'MateBook D', 'MateBook X Pro'], 'Asus': ['ExpertBook', 'ProArt', 'V16', 'Vivobook', 'Vivobook 15', 'Vivobook Go', 'Vivobook Pro 15', 'Vivobook S', 'Zenbook', 'Zenbook DUO'], 'Apple': ['MacBook Air', 'MacBook Air 13', 'MacBook Air 13 M1', 'MacBook Air 15 M2', 'MacBook Pro', 'MacBook Pro 13', 'MacBook Pro 13 M1', 'MacBook Pro 13 M2', 'MacBook Pro 14 M1', 'MacBook Pro 14 M2 Max', 'MacBook Pro 16 M1'], 'HP': ['EliteBook 840', 'Notebook', 'OmniBook', 'Pavilion', 'Pavilion Plus', 'Victus'], 'Honor': ['MagicBook Pro 14'], 'MSI': ['Modern', 'Moder